<a href="https://colab.research.google.com/github/prinn10/autoNarajangteo/blob/main/%EB%82%98%EB%9D%BC%EC%9E%A5%ED%84%B0_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount and Import Module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

path='/content/drive/MyDrive/'
name='dataset_result.csv'

In [3]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

import seaborn as sns
import warnings
import matplotlib as mpl

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Call Dataset and Preprocessing


In [4]:
#df=pd.read_csv(path+name, encoding='euc-kr')
df=pd.read_csv(path+name, encoding='utf-8-sig')

print(df)

df_shuffled=df.sample(frac=1).reset_index(drop=True)
print(df_shuffled[:-200])
print(df_shuffled[-200:])

              입찰공고번호         기초금액         예정가격  참여업체수   낙찰하한율  기초금액기준상위개수  \
0     20171238310-00  345180000.0  343965000.0     18  87.995           7   
1     20171235590-00   81125000.0   80944150.0      2  88.000           8   
2     20171238233-00  368420000.0  369927825.0      5  80.495           8   
3     20171238707-00   78898000.0   79020750.0     23  87.995           7   
4     20171235088-00   56800000.0   56403700.0      2  87.995           7   
...              ...          ...          ...    ...     ...         ...   
1242  20220501644-00   39600000.0   39698650.0     73  88.000           7   
1243  20220501656-00     100000.0     100025.0      4  88.000           7   
1244  20220501669-00  103915823.0  102746550.0   5280  84.245           8   
1245  20220501787-00   15378000.0   15466400.0     71  88.000           8   
1246  20220501806-00   12041280.0   11877175.0     46  84.245           7   

             투찰금액     투찰률  예가범위1  예가범위2  target rate  
0     303194407.0  8

In [5]:
from sklearn.preprocessing import MinMaxScaler
def preprocess(df):
  PC_df = df[['예정가격']]
  q1_df = df[['예가범위1']].astype('float32')
  q2_df = df[['예가범위2']].astype('float32')
  bidlimit_df = df[['낙찰하한율']]

  print(df[['기초금액']].shape, PC_df.shape, q1_df.shape, q2_df.shape, bidlimit_df.shape)

  BC = df.loc[:, ['기초금액']].to_numpy()
  PC = df.loc[:, ['예정가격']].to_numpy()
  q1 = df.loc[:, ['예가범위1']].astype('float32').to_numpy()
  q2 = df.loc[:, ['예가범위2']].astype('float32').to_numpy()
  bidlimit = df.loc[:, ['낙찰하한율']].to_numpy()

  print(len(BC), len(PC), len(q1), len(q2), len(bidlimit))

  for i in range(len(BC)):
      pb = PC[i] / BC[i]
      totrange = q1[i]+q2[i]
      ratio = (100*pb-100+q1[i]) / totrange
      PC[i] = ratio
      q1[i] /= 100.0
      q2[i] /= 100.0
      bidlimit[i] /= 100.0

  PC_df = pd.DataFrame(PC, columns=['예정가격'])
  q1_df = pd.DataFrame(q1, columns=['예가범위1'])
  q2_df = pd.DataFrame(q2, columns=['예가범위2'])
  bidlimit_df = pd.DataFrame(bidlimit, columns=['낙찰하한율'])
    
  scaler = MinMaxScaler()
  scale_cols = ['기초금액', '기초금액기준상위개수', '투찰률'] #정규화 대상 column 정의
  scaled_df = scaler.fit_transform(df[scale_cols]) #정규화 수행
  print(type(scaled_df), '\n') #리턴 값은 넘파이
  scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)
  notminmaxdf = pd.concat([pd.concat([PC_df, bidlimit_df], axis=1), pd.concat([q1_df, q2_df], axis=1)], axis=1)
  Final_df = pd.concat([scaled_df, notminmaxdf], axis=1)
  feature_cols = ['기초금액', '낙찰하한율', '기초금액기준상위개수', '예가범위1', '예가범위2']
  label_cols = ['예정가격']
  feature_df = pd.DataFrame(Final_df, columns=feature_cols)
  label_df = pd.DataFrame(Final_df, columns=label_cols)
  x, y = torch.from_numpy(feature_df.to_numpy()), torch.from_numpy(label_df.to_numpy())
  ds_test=TensorDataset(x, y)
  return ds_test


In [6]:
df=pd.read_csv(path+name, encoding='utf-8-sig')
df.head()
del df['target rate']

In [7]:
col_name = list(df.keys())

In [8]:
for tmp in col_name:
    #print(df.loc[df[tmp].isnull()])
    drop_idx = df.loc[df[tmp].isnull()].index
    df= df.drop(drop_idx)

In [9]:
for tmp in col_name:
    df.loc[df[tmp].isnull()]

In [10]:
#예정가격, 낙찰하한율, 예가범위 전처리
PC_df = df[['예정가격']]
q1_df = df[['예가범위1']].astype('float32')
q2_df = df[['예가범위2']].astype('float32')
bidlimit_df = df[['낙찰하한율']]

print(df[['기초금액']].shape, PC_df.shape, q1_df.shape, q2_df.shape, bidlimit_df.shape)

BC = df.loc[:, ['기초금액']].to_numpy()
PC = df.loc[:, ['예정가격']].to_numpy()
q1 = df.loc[:, ['예가범위1']].astype('float32').to_numpy()
q2 = df.loc[:, ['예가범위2']].astype('float32').to_numpy()
bidlimit = df.loc[:, ['낙찰하한율']].to_numpy()

print(len(BC), len(PC), len(q1), len(q2), len(bidlimit))

for i in range(len(BC)):
    pb = PC[i] / BC[i]
    totrange = q1[i]+q2[i]
    ratio = (100*pb-100+q1[i]) / totrange
    PC[i] = ratio
    q1[i] /= 100.0
    q2[i] /= 100.0
    bidlimit[i] /= 100.0

# for i in range(len(BC)):
#     PC_df.loc[i] = PC[i]
#     q1_df.loc[i] = q1[i]
#     q2_df.loc[i] = q2[i]
#     bidlimit_df.loc[i] = bidlimit[i]

PC_df = pd.DataFrame(PC, columns=['예정가격'])
q1_df = pd.DataFrame(q1, columns=['예가범위1'])
q2_df = pd.DataFrame(q2, columns=['예가범위2'])
bidlimit_df = pd.DataFrame(bidlimit, columns=['낙찰하한율'])

print(PC_df.shape, q1_df.shape, q2_df.shape, bidlimit_df.shape)

(1247, 1) (1247, 1) (1247, 1) (1247, 1) (1247, 1)
1247 1247 1247 1247 1247
(1247, 1) (1247, 1) (1247, 1) (1247, 1)


In [11]:
# 정규화 (Date 제외한 모든 수치부분 정규화)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scale_cols = ['기초금액', '기초금액기준상위개수', '투찰률'] #정규화 대상 column 정의

scaled_df = scaler.fit_transform(df[scale_cols]) #정규화 수행
print(type(scaled_df), '\n') #리턴 값은 넘파이

scaled_df = pd.DataFrame(scaled_df, columns=scale_cols) #정규화된 새로운 DataFrame 생성

print(scaled_df)

<class 'numpy.ndarray'> 

          기초금액  기초금액기준상위개수       투찰률
0     0.060043    0.272727  0.642622
1     0.014111    0.363636  0.984256
2     0.064085    0.363636  0.687559
3     0.013724    0.272727  0.639932
4     0.009880    0.272727  0.656946
...        ...         ...       ...
1242  0.006888    0.272727  0.638754
1243  0.000017    0.272727  0.648364
1244  0.018076    0.363636  0.525067
1245  0.002675    0.363636  0.638331
1246  0.002095    0.272727  0.527636

[1247 rows x 3 columns]


In [12]:

notminmaxdf = pd.concat([pd.concat([PC_df, bidlimit_df], axis=1), pd.concat([q1_df, q2_df], axis=1)], axis=1)
notminmaxdf.head()

,예정가격,낙찰하한율,예가범위1,예가범위2
0,0.441335,0.87995,0.03,0.03
1,0.444268,0.88000,0.02,0.02
2,0.602317,0.80495,0.02,0.02
3,0.525930,0.87995,0.03,0.03
4,0.383715,0.87995,0.03,0.03


In [13]:

Final_df = pd.concat([scaled_df, notminmaxdf], axis=1)
Final_df.head()



,기초금액,기초금액기준상위개수,투찰률,예정가격,낙찰하한율,예가범위1,예가범위2
0,0.060043,0.272727,0.642622,0.441335,0.87995,0.03,0.03
1,0.014111,0.363636,0.984256,0.444268,0.88000,0.02,0.02
2,0.064085,0.363636,0.687559,0.602317,0.80495,0.02,0.02
3,0.013724,0.272727,0.639932,0.525930,0.87995,0.03,0.03
4,0.009880,0.272727,0.656946,0.383715,0.87995,0.03,0.03


In [14]:
tmp='기초금액'
Final_df.loc[Final_df[tmp].isnull()]

,기초금액,기초금액기준상위개수,투찰률,예정가격,낙찰하한율,예가범위1,예가범위2


# 회귀 분석 및 분산 분석

In [15]:
# feature_df, label_df 생성

feature_cols = ['기초금액', '낙찰하한율', '기초금액기준상위개수', '예가범위1', '예가범위2']
label_cols = ['예정가격']

feature_df = pd.DataFrame(Final_df, columns=feature_cols)
label_df = pd.DataFrame(Final_df, columns=label_cols)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feature_df, label_df, train_size=0.8, test_size=0.2)
x_train.head()

,기초금액,낙찰하한율,기초금액기준상위개수,예가범위1,예가범위2
88,3.359807e-02,0.84245,0.272727,0.03,0.03
147,2.539147e-07,0.84245,0.272727,0.03,0.03
180,6.195503e-03,0.88000,0.272727,0.02,0.02
477,1.033241e-02,0.88000,0.272727,0.03,0.03
1023,5.992451e-02,0.80495,0.363636,0.02,0.02


In [17]:
model = sm.OLS(y_train, sm.add_constant(x_train))
results = model.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [18]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   예정가격   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.226
Date:                Fri, 27 May 2022   Prob (F-statistic):             0.0497
Time:                        03:06:45   Log-Likelihood:                 639.37
No. Observations:                 997   AIC:                            -1267.
Df Residuals:                     991   BIC:                            -1237.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8821      0.149      5.909      0.000       0.589       1.175
기초금액          -0.0440      0.073     -0.606      0.545      -0.187       0.099
낙찰하한율         -0.4263      0.160     -2.662      0.008      -0.741      -0.112
기초금액기준상위개수     0.0233      0.140      0.166      0.868      -0.252       0.298
예가범위1          2.1513      9.736      0.221      0.825     -16.955      21.257
예가범위2         -3.3585      9.412     -0.357      0.721     -21.828      15.111
==============================================================================
Omnibus:                        4.991   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.082   Jarque-Bera (JB):                3.875
Skew:                           0.000   Prob(JB):                        0.144
Kurtosis:                       2.695   Cond. No.                     4.53e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.53e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
formula = '예정가격~기초금액+낙찰하한율+기초금액기준상위개수+예가범위1+예가범위2'
df_anova = pd.concat([x_train, y_train], axis=1)
model_anova = sm.OLS.from_formula(formula, data=df_anova)

results_anova = model_anova.fit()
sm.stats.anova_lm(results_anova, typ=2)

,sum_sq,df,F,PR(>F)
기초금액,0.005991,1.0,0.366755,0.544917
낙찰하한율,0.115724,1.0,7.084252,0.007902
기초금액기준상위개수,0.000450,1.0,0.027555,0.868194
예가범위1,0.000798,1.0,0.048822,0.825171
예가범위2,0.002080,1.0,0.127331,0.721292
Residual,16.188332,991.0,NaN,NaN


# RandomForest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

#x_train, x_test, y_train, y_test = train_test_split(X_st, Y_st, train_size=0.8, test_size=0.2)

param_grid={
    'n_estimators':(50, 100, 150),
    'min_samples_leaf' : (2, 8, 18),
    'min_samples_split' : (1, 8, 16)
}
regressor = RandomForestRegressor()
grid_search=GridSearchCV(regressor, param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/par

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'min_samples_leaf': (2, 8, 18),
                         'min_samples_split': (1, 8, 16),
                         'n_estimators': (50, 100, 150)})

In [22]:
print('최적 하이퍼 파라미터:', grid_search.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_search.best_score_))

최적 하이퍼 파라미터: {'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 50}
최적 예측 정확도: -0.0329


In [23]:
regressor = RandomForestRegressor(n_estimators = 50, min_samples_leaf = 8, min_samples_split = 8)
regressor.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(min_samples_leaf=8, min_samples_split=8, n_estimators=50)

In [24]:
# train rmse
train_predict = regressor.predict(x_train)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, y_train))) )

RMSE':0.11556083675541062


In [25]:
test_predict = regressor.predict(x_test)
print("RMSE':{}".format(math.sqrt(mean_squared_error(test_predict, y_test))) )

for i in range(len(test_predict)):
    print(test_predict[i], y_test.iloc[i])

RMSE':0.1390281249777913
0.5082749226550137 예정가격    0.378
Name: 368, dtype: float64
0.5214509865392928 예정가격    0.485813
Name: 1123, dtype: float64
0.5079548776968961 예정가격    0.6075
Name: 263, dtype: float64
0.49839632961134905 예정가격    0.472
Name: 198, dtype: float64
0.4751524512265868 예정가격    0.501007
Name: 1004, dtype: float64
0.4258995261883428 예정가격    0.26617
Name: 854, dtype: float64
0.41003722078364474 예정가격    0.725391
Name: 882, dtype: float64
0.4478761190845183 예정가격    0.450232
Name: 474, dtype: float64
0.5082749226550137 예정가격    0.273583
Name: 1215, dtype: float64
0.4814659191552703 예정가격    0.546535
Name: 460, dtype: float64
0.4769131020669209 예정가격    0.494061
Name: 525, dtype: float64
0.47599955269743965 예정가격    0.282073
Name: 1009, dtype: float64
0.4844324412494515 예정가격    0.552838
Name: 934, dtype: float64
0.5026068432035783 예정가격    0.613084
Name: 784, dtype: float64
0.5362597556524039 예정가격    0.297012
Name: 756, dtype: float64
0.5304791455656716 예정가격    0.25681
Name: 520, d

# DNN

In [26]:
# feature_df, label_df 생성

feature_cols = ['기초금액', '낙찰하한율', '기초금액기준상위개수', '예가범위1', '예가범위2']
label_cols = ['예정가격', '투찰률']

feature_df = pd.DataFrame(Final_df, columns=feature_cols)
label_df = pd.DataFrame(Final_df, columns=label_cols)

In [27]:
from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(feature_df, label_df, train_size=0.8, test_size=0.2)
# x_test.head()
x_train, y_train = torch.from_numpy(feature_df.to_numpy()), torch.from_numpy(label_df.to_numpy())

In [28]:
import torch

In [29]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [30]:
def weight_init(submodule):
    if isinstance(submodule, nn.Linear):
        nn.init.kaiming_normal_(submodule.weight)
        submodule.bias.data.fill_(0.01)

def layer(input_size, output_size):
    return nn.Sequential(
            nn.Linear(input_size, output_size, bias=True),
            nn.ReLU(),
            #nn.Dropout(0.1)
        )

class DNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DNN, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size

        self.layer1=nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size[0], bias=True),
            nn.ReLU(),
            #nn.Dropout(0.1)
        )

        self.hidden_layers=nn.ModuleList([layer(hidden_size[i], hidden_size[i+1]) for i in range(len(hidden_size)-1)])
        self.fc=nn.Linear(hidden_size[len(hidden_size)-1], output_size, bias=True)

        
        self.layer1.apply(weight_init)
        
        for Layer in self.hidden_layers:
            Layer.apply(weight_init)
        
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):
        out=self.layer1(x)
        
        for Layer in self.hidden_layers:
            out=Layer(out)

        out=self.fc(out)
        return out


In [31]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [32]:
# x_train, y_train, x_test, y_test = torch.from_numpy(x_train.to_numpy()), torch.from_numpy(y_train.to_numpy()), torch.from_numpy(x_test.to_numpy()), torch.from_numpy(y_test.to_numpy()), 

In [33]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# 1. df 로드
name='dataset_result.csv'
# df=pd.read_csv(path+name, encoding='euc-kr')
df=pd.read_csv(path+name, encoding='utf-8-sig')
df=df.sample(frac=1).reset_index(drop=True)

# 2. 전처리 및 Dataset 인스턴스 생성
del df['target rate']
ds_train = preprocess(df[:-200])
ds_test = preprocess(df[-200:])

# 3. DataLoader 인스턴스 생성
loader_train=DataLoader(ds_train, batch_size=16, shuffle=True)

# ds_train=TensorDataset(x_train, y_train)
# # ds_test=TensorDataset(x_test, y_test)

# loader_train=DataLoader(ds_train, batch_size=16, shuffle=True)
# # loader_test=DataLoader(ds_test, batch_size=16, shuffle=False)

learning_rate=0.0001
total_epochs=200

HL_list=[12, 6]

model=DNN(5, HL_list, 2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

criterion = RMSELoss().to(device)

(1047, 1) (1047, 1) (1047, 1) (1047, 1) (1047, 1)
1047 1047 1047 1047 1047
<class 'numpy.ndarray'> 

(200, 1) (200, 1) (200, 1) (200, 1) (200, 1)
200 200 200 200 200
<class 'numpy.ndarray'> 



In [34]:
total_batch = len(loader_train)
# val_total_batch = len(loader_test)
print(total_batch)

for epoch in range(total_epochs):

    avg_loss=0

    best_val_loss = 10000
    for X, Y in loader_train:
        #LRprediction = modelfunction(X)
        X=X.float().to(device)
        Y=Y.float().to(device)
        #LRprediction = LRprediction.float().to(device)


        # if epoch ==0:
        #     print(X)

        optimizer.zero_grad()
        pred = model(X)

        # if epoch %10 ==0:
        #     print(pred, Y)
        loss = criterion(pred, Y)
        loss.backward()
        optimizer.step()

        avg_loss += loss / total_batch
    
    with torch.no_grad():
        val_loss = 0
        # for X, Y in loader_test:
            
        #     X=X.float().to(device)
        #     Y=Y.float().to(device)

        #     pred = model(X)

        #     loss = criterion(pred, Y)

        #     val_loss += loss / val_total_batch

    if epoch % 10 == 0:
        print('Epoch: {} train_loss = {}, val_loss = {}'.format(epoch+1, avg_loss, val_loss))



66


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1 train_loss = 1.1186093091964722, val_loss = 0
Epoch: 11 train_loss = 0.5412943959236145, val_loss = 0
Epoch: 21 train_loss = 0.44105294346809387, val_loss = 0
Epoch: 31 train_loss = 0.3731015920639038, val_loss = 0
Epoch: 41 train_loss = 0.31823647022247314, val_loss = 0
Epoch: 51 train_loss = 0.2722826898097992, val_loss = 0
Epoch: 61 train_loss = 0.23196116089820862, val_loss = 0
Epoch: 71 train_loss = 0.19836100935935974, val_loss = 0
Epoch: 81 train_loss = 0.1685347557067871, val_loss = 0
Epoch: 91 train_loss = 0.14612551033496857, val_loss = 0
Epoch: 101 train_loss = 0.1353328675031662, val_loss = 0
Epoch: 111 train_loss = 0.13012486696243286, val_loss = 0
Epoch: 121 train_loss = 0.12844263017177582, val_loss = 0
Epoch: 131 train_loss = 0.1276625096797943, val_loss = 0
Epoch: 141 train_loss = 0.12826618552207947, val_loss = 0
Epoch: 151 train_loss = 0.12776139378547668, val_loss = 0
Epoch: 161 train_loss = 0.12860751152038574, val_loss = 0
Epoch: 171 train_loss = 0.127850

In [35]:
# 기초금액	참여업체수 낙찰하한율 기초금액기준상위개수 예가범위1 예가범위2
# 예정가격
def reverse_PC(x_test, pred, BC):
  totrange = (x_test[3]*100)+(x_test[4]*100)
  return ((pred * totrange + 100 - (x_test[3]*100)) * BC / 100)

def bid_res(pred_PC, sPC, smin_rate, sBidCost, ann_num): #낙찰 여부를 판독하는 함수
  # 투찰가
  BidCost = float(sBidCost)
  # 낙찰하한가MinCost pred_PC
  MinCost = float(sPC) * float(smin_rate) /100
  pred_PC = float(pred_PC) * float(smin_rate) /100
  print('입찰공고번호', ann_num, '투찰가', round(BidCost, 1), '낙찰하한가', round(MinCost, 1), '예측 가격', round(pred_PC, 1), '예정가격', sPC, '낙찰하한율', smin_rate, end='')
  if pred_PC < BidCost and pred_PC >= MinCost:
    print('TP')
    return True
  else:
    print('TN')
    return False


In [36]:
total_len = len(ds_test)
TP = 0
TF = 0
for idx in range(len(ds_test)):

    x_test, y_test = ds_test[idx]

    pred = model(x_test.float().to(device)).detach().cpu().numpy()
    pred_PC = reverse_PC(x_test.detach().cpu().numpy(), pred[0], float(df['기초금액'][idx]))
    if bid_res(pred_PC, df['예정가격'][idx], df['낙찰하한율'][idx], df['투찰금액'][idx], df['입찰공고번호'][idx]):
      TP += 1
    else:
      TF += 1

print(total_len, TP, TF)

입찰공고번호 20220505083-00 투찰가 190854730.0 낙찰하한가 189992308.3 예측 가격 189469269.5 예정가격 236029950.0 낙찰하한율 80.495TN
입찰공고번호 20220513309-00 투찰가 429.0 낙찰하한가 425.5 예측 가격 431.9 예정가격 472.8 낙찰하한율 90.0TN
입찰공고번호 20220503170-00 투찰가 32451900.0 낙찰하한가 32449670.0 예측 가격 32543432.7 예정가격 36874625.0 낙찰하한율 88.0TN
입찰공고번호 20220519109-01 투찰가 19500000.0 낙찰하한가 18554206.0 예측 가격 18408516.1 예정가격 21084325.0 낙찰하한율 88.0TN
입찰공고번호 20220443128-00 투찰가 42900000.0 낙찰하한가 35556680.0 예측 가격 35653124.1 예정가격 44445850.0 낙찰하한율 80.0TP
입찰공고번호 20220516056-00 투찰가 76626190.0 낙찰하한가 70349840.0 예측 가격 70260254.1 예정가격 79943000.0 낙찰하한율 88.0TN
입찰공고번호 20220504055-00 투찰가 26020070.0 낙찰하한가 26018520.0 예측 가격 26026830.2 예정가격 29566500.0 낙찰하한율 88.0TN
입찰공고번호 20220500149-00 투찰가 23055000.0 낙찰하한가 22809182.0 예측 가격 22972453.2 예정가격 25919525.0 낙찰하한율 88.0TP
입찰공고번호 20220512913-00 투찰가 66027000.0 낙찰하한가 57747293.8 예측 가격 57656611.9 예정가격 68546850.0 낙찰하한율 84.245TN
입찰공고번호 20220511573-00 투찰가 91564774.0 낙찰하한가 91528360.0 예측 가격 92256234.0 예정가격 104009500.0 낙찰하한율 88.0TN
입찰공고번호 2022

# LSTM

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
raw_df = pd.read_csv(path+'manhattan.csv')  

raw_df.head()

In [ ]:
# 정규화 (Date 제외한 모든 수치부분 정규화)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scale_cols = ['bedrooms', 'bathrooms', 'size_sqft', 'min_to_subway', 'floor',
              'building_age_yrs', 'rent'] #정규화 대상 column 정의

scaled_df = scaler.fit_transform(raw_df[scale_cols]) #정규화 수행
print(type(scaled_df), '\n') #리턴 값은 넘파이

scaled_df = pd.DataFrame(scaled_df, columns=scale_cols) #정규화된 새로운 DataFrame 생성

print(scaled_df)

In [ ]:

# feature_df, label_df 생성

feature_cols = ['bedrooms', 'bathrooms', 'size_sqft', 'min_to_subway', 'floor',
              'building_age_yrs', 'rent']
label_cols = [ 'rent' ]

feature_df = pd.DataFrame(scaled_df, columns=feature_cols)
label_df = pd.DataFrame(scaled_df, columns=label_cols)
print(feature_df)
print(label_df)


# DataFrame => Numpy 변환

feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

print(feature_np.shape, label_np.shape)

In [ ]:
# 입력 파라미터 feature, label => numpy type

def make_sequene_dataset(feature, label, window_size):

    feature_list = []      # 생성될 feature list
    label_list = []        # 생성될 label list

    for i in range(len(feature)-window_size):

        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])

    return np.array(feature_list), np.array(label_list) #3차원 텐서로 변환

In [ ]:
window_size = 100

X, Y = make_sequene_dataset(feature_np, label_np, window_size)

print(X.shape, Y.shape)

In [ ]:
# train, test 분리

#split = int(len(X)*0.95)
split = -200

x_train = X[0:split]
y_train = Y[0:split]

x_test = X[split:]
y_test = Y[split:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# model 생성

model = Sequential()

model.add(LSTM(200, activation='tanh', input_shape=x_train[0].shape))

model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=15)

model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=100, batch_size=32,
          callbacks=[early_stop])

In [ ]:
pred = model.predict(x_test)

plt.figure(figsize=(12, 6))
plt.title('bedrooms+bathrooms+size_sqft+min_to_subway+floor+building_age_yrs+rent, window_size=40')
plt.ylabel('rent')
plt.xlabel('period')
plt.plot(y_test, label='actual')
plt.plot(pred, label='prediction')
plt.grid()
plt.legend(loc='best')

plt.show()

In [ ]:
import torch
import torch.nn as nn
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self, x, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [ ]:
print( np.sum((y_test-pred)**2) / len(x_test) )

In [ ]:
print( np.sum(abs(y_test-pred)/y_test) / len(x_test) )

# Test

In [ ]:
for idx in range(len(ds_test)):

    x_test, y_test = ds_test[idx]

    pred = model(x_test.float().to(device))

    print("pred: ", pred)
    print("label: ", y_test)